# Milestone 2: Model Evaluation & Ranking Generation
**Mục tiêu:**
1. Đọc dữ liệu `labels.json` (chứa input BibTeX và Ground Truth).
2. Tái cấu trúc dữ liệu theo từng bài báo (Paper-based grouping).
3. Sử dụng mô hình đã huấn luyện để xếp hạng (Ranking) các reference candidates.
4. Tính chỉ số **MRR (Mean Reciprocal Rank)**.
5. Xuất file `pred.json` theo đúng định dạng yêu cầu nộp bài.

**Yêu cầu đầu vào:**
* `labels.json`: File dữ liệu test.
* `models/best_matcher.pkl`: Model đã train.
* `models/feature_names.pkl`: List tên features tương ứng.


In [64]:
import pandas as pd
import numpy as np
import json
import os
import re
import joblib
from tqdm import tqdm
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import warnings
warnings.filterwarnings('ignore')

# Cài đặt hiển thị
pd.set_option('display.max_columns', None)
print("Libraries imported successfully!")


Libraries imported successfully!


In [65]:
# --- CẤU HÌNH ĐƯỜNG DẪN ---
# Hãy thay đổi đường dẫn phù hợp với môi trường của bạn
TEST_FILE_PATH = '../../dataset_final/test/labels.json'           # File dữ liệu test
MODEL_PATH = '../../dataset_final/models/best_matcher.pkl'   # File model (đã train ở bước trước)
FEATURE_NAME_PATH = '../../dataset_final/models/feature_names.pkl' # Danh sách features
OUTPUT_DIR = 'submission_output'         # Thư mục chứa kết quả pred.json

if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"Created output directory: {OUTPUT_DIR}")


In [66]:

def normalize_text_basic(text):
    """Chuẩn hóa cơ bản để tính toán khoảng cách"""
    if not isinstance(text, str): return ""
    return str(text).lower().strip()

def get_tokens(text_list_or_str):
    """Chuyển đổi text hoặc list text thành set các từ đơn (tokens)"""
    if isinstance(text_list_or_str, list):
        text = " ".join([str(t) for t in text_list_or_str])
    else:
        text = str(text_list_or_str)
    
    # Bỏ dấu câu, giữ lại chữ số và chữ cái
    text = re.sub(r'[^\w\s]', '', text.lower())
    return set(text.split())

def safe_year_diff(y1, y2):
    """Tính khoảng cách năm, xử lý lỗi nếu thiếu dữ liệu"""
    try:
        # Lấy 4 số đầu tiên tìm thấy làm năm
        m1 = re.search(r'\d{4}', str(y1))
        m2 = re.search(r'\d{4}', str(y2))
        
        if m1 and m2:
            val1 = int(m1.group(0))
            val2 = int(m2.group(0))
            diff = abs(val1 - val2)
            # Clip khoảng cách để tránh outlier quá lớn (ví dụ sai số 100 năm)
            return min(diff, 10) 
        return -1 # Giá trị missing indicator
    except:
        return -1


In [67]:
# --- 1. PARSER THÔNG MINH V3 (COPY TỪ NOTEBOOK 01) ---
import bibtexparser
from pylatexenc.latex2text import LatexNodes2Text
import re

latex_decoder = LatexNodes2Text()
P_YEAR_KEY = re.compile(r'year\s*=\s*[\{\"]?\s*([12]\d{3})\s*[\}\"]?', re.IGNORECASE)
P_YEAR_SIMPLE = re.compile(r'\b(19|20)\d{2}\b')

def clean_latex(text):
    if not isinstance(text, str) or not text: return ""
    try:
        text = latex_decoder.latex_to_text(text)
        text = text.replace('{', '').replace('}', '')
        text = " ".join(text.split())
        return text
    except: return text

def normalize_id(text):
    if not text: return ""
    return re.sub(r'[\.\-\/]', '', str(text).lower())

def parse_bibtex_smart(bib_string):
    """Parser V3: Lấy đủ Title, Author, Year, ID"""
    if not bib_string: return {}, "empty"
    parsed_data = {"title": "", "authors": [], "year": "", "extracted_id": ""}
    
    # 1. Tìm năm (Ưu tiên)
    year_match = P_YEAR_KEY.search(bib_string)
    if year_match: parsed_data['year'] = year_match.group(1)

    # 2. Parse BibTeX Struct
    if bib_string.strip().startswith('@'):
        try:
            parser = bibtexparser.bparser.BibTexParser(common_strings=True)
            bib_db = bibtexparser.loads(bib_string, parser=parser)
            if bib_db.entries:
                entry = bib_db.entries[0]
                parsed_data['title'] = clean_latex(entry.get('title', ''))
                raw_authors = entry.get('author', '')
                if raw_authors: parsed_data['authors'] = [clean_latex(a.strip()) for a in raw_authors.split(' and ')]
                raw_id = entry.get('eprint', entry.get('doi', ''))
                parsed_data['extracted_id'] = normalize_id(raw_id)
                if not parsed_data['year']: parsed_data['year'] = entry.get('year', '')
                return parsed_data
        except: pass

    # 3. Parse Flat Text
    if r'\newblock' in bib_string:
        try:
            parts = bib_string.split(r'\newblock')
            raw_author_str = parts[0].strip()
            if raw_author_str.endswith('.'): raw_author_str = raw_author_str[:-1]
            parsed_data['authors'] = [clean_latex(a.strip()) for a in raw_author_str.split(',')]
            if len(parts) > 1: parsed_data['title'] = clean_latex(parts[1].strip())
            if not parsed_data['year']:
                simple_match = P_YEAR_SIMPLE.search(parts[-1])
                if simple_match: parsed_data['year'] = simple_match.group(0)
            return parsed_data
        except: pass
            
    # 4. Fallback
    if not parsed_data['title']:
        tm = re.search(r'title\s*=\s*[\{"](.*?)(?<!\\)[\}"]', bib_string, re.I | re.S)
        if tm: parsed_data['title'] = clean_latex(tm.group(1))
    if not parsed_data['year']:
        ym = P_YEAR_SIMPLE.search(bib_string)
        if ym: parsed_data['year'] = ym.group(0)

    return parsed_data

## 1. Feature Engineering
**Lưu ý quan trọng:** Các hàm dưới đây phải **giống hệt** logic bạn đã sử dụng khi huấn luyện mô hình (Training Phase). Nếu thay đổi, model sẽ nhận diện sai đặc trưng.


In [68]:
# --- Helper: Text Processing ---
def normalize_text(text):
    if pd.isna(text) or text is None: return ""
    text = str(text).lower().strip()
    return text

def get_tokens(text_list_or_str):
    if isinstance(text_list_or_str, list):
        text = " ".join([str(t) for t in text_list_or_str])
    else:
        text = str(text_list_or_str)
    text = re.sub(r'[^\w\s]', '', text.lower())
    return set(text.split())
def parse_bibtex_content(bib_content):
    """
    Trích xuất Title và Authors từ chuỗi BibTeX raw.
    Đã sửa lỗi Regex pattern error.
    """
    # Extract Title
    # SỬA LỖI: Thay (?<!\) thành (?<!\\)
    title_match = re.search(r'title\s*=\s*[\{"](.*?)(?<!\\)[\}"]', bib_content, re.I | re.S)
    title = title_match.group(1) if title_match else ""
    title = normalize_text(title)
    
    # Extract Authors
    # SỬA LỖI: Tương tự cho phần authors
    author_match = re.search(r'author\s*=\s*[\{"](.*?)(?<!\\)[\}"]', bib_content, re.I | re.S)
    authors_str = author_match.group(1) if author_match else ""
    # Tách author đơn giản bằng 'and'
    authors = [normalize_text(a) for a in authors_str.split(' and ')] if authors_str else []
    
    return title, authors

In [69]:
# --- 2. FEATURE ENGINEERING (FULL) ---
def compute_pairwise_features(row):
    feats = {}
    
    # UNPACK
    q_tit = normalize_text_basic(row.get('bib_title', ''))
    q_auth_list = row.get('bib_authors', [])
    q_id = normalize_text_basic(row.get('bib_id', ''))
    q_year = str(row.get('bib_year', '')) # Đã có năm
    
    c_tit = normalize_text_basic(row.get('cand_title', ''))
    c_auth_list = row.get('cand_authors', [])
    c_id = normalize_text_basic(row.get('cand_id', ''))
    c_year = str(row.get('cand_year', '')) # Đã có năm
    
    q_auth_str = " ".join(q_auth_list) if isinstance(q_auth_list, list) else str(q_auth_list)
    c_auth_str = " ".join(c_auth_list) if isinstance(c_auth_list, list) else str(c_auth_list)

    # 1. ID MATCH
    id_score = 0.0
    if q_id and c_id:
        clean_q = re.sub(r'[^a-z0-9]', '', q_id)
        clean_c = re.sub(r'[^a-z0-9]', '', c_id)
        if clean_q == clean_c: id_score = 1.0
        elif clean_q in clean_c or clean_c in clean_q: id_score = 0.8
    feats['feat_id_match'] = id_score

    # 2. TITLE FEATURES
    feats['feat_title_fuzzy'] = fuzz.ratio(q_tit, c_tit) / 100.0
    feats['feat_title_sort'] = fuzz.token_sort_ratio(q_tit, c_tit) / 100.0
    feats['feat_title_partial'] = fuzz.partial_ratio(q_tit, c_tit) / 100.0
    feats['feat_title_contain'] = 1.0 if (q_tit and c_tit and (q_tit in c_tit or c_tit in q_tit)) else 0.0
    
    len_q, len_c = len(q_tit), len(c_tit)
    feats['feat_title_len_diff'] = abs(len_q - len_c) / max(len_q, len_c, 1)

    # [cite_start]3. TF-IDF COSINE SIMILARITY (CỰC KỲ QUAN TRỌNG) [cite: 56]
    # Vì ta không load vectorizer toàn cục, ta tính "Local Cosine" cho cặp này
    # Dùng TfidfVectorizer fit trên 2 câu này thôi cũng tạo ra được score khác 0
    try:
        if q_tit and c_tit:
            vec = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
            tfidf = vec.fit_transform([q_tit, c_tit])
            # cosine của vector 0 và vector 1
            feats['feat_title_tfidf_cosine'] = cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0]
        else:
            feats['feat_title_tfidf_cosine'] = 0.0
    except:
        feats['feat_title_tfidf_cosine'] = 0.0

    # 4. AUTHOR FEATURES
    q_tokens = get_tokens(q_auth_list)
    c_tokens = get_tokens(c_auth_list)
    
    if q_tokens and c_tokens:
        inter = len(q_tokens.intersection(c_tokens))
        union = len(q_tokens.union(c_tokens))
        feats['feat_auth_jaccard'] = inter / union
        feats['feat_auth_overlap'] = inter
    else:
        feats['feat_auth_jaccard'] = 0.0
        feats['feat_auth_overlap'] = 0
        
    feats['feat_auth_token_sort'] = fuzz.token_sort_ratio(q_auth_str, c_auth_str) / 100.0
    
    try:
        a1_q = str(q_auth_list[0]).split()[0].lower() if len(q_auth_list) > 0 else ""
        a1_c = str(c_auth_list[0]).split()[0].lower() if len(c_auth_list) > 0 else ""
        feats['feat_first_auth_match'] = 1.0 if (a1_q and a1_c and a1_q[:3] == a1_c[:3]) else 0.0
    except:
        feats['feat_first_auth_match'] = 0.0
        
    # 5. YEAR FEATURE
    feats['feat_year_diff'] = safe_year_diff(q_year, c_year)
    feats['feat_year_match'] = 1.0 if (feats['feat_year_diff'] == 0) else 0.0

    return feats

## 2. Load & Transform Data
Chúng ta cần chuyển đổi dữ liệu từ danh sách phẳng (flat list) trong `manual.json` sang cấu trúc **Paper-based**.
* Mỗi `source_paper_id` sẽ là một nhóm.
* Tập **Candidates** của nhóm đó là tập hợp tất cả các `ground_truth` unique xuất hiện trong paper.


In [70]:
# --- 3. DATA LOADING & TRANSFORMATION ---
try:
    with open(TEST_FILE_PATH, 'r', encoding='utf-8') as f:
        raw_data = json.load(f)
    print(f"Loaded {len(raw_data)} entries.")
except FileNotFoundError:
    print("❌ Error: File not found.")
    raw_data = []

papers_db = defaultdict(lambda: {'queries': [], 'candidates': {}})

for item in raw_data:
    paper_id = item.get('source_paper_id', 'unknown_paper')
    
    # 1. Input Query (Dùng Parser V3)
    bib_key = item.get('key')
    bib_content = item.get('content')
    
    # Parse lấy cả Year
    p_data = parse_bibtex_smart(bib_content)
    
    query_obj = {
        'key': bib_key,
        'bib_title': p_data['title'],
        'bib_authors': p_data['authors'],
        'bib_id': p_data['extracted_id'],
        'bib_year': p_data['year'], # <--- ĐÃ CÓ NĂM
        'true_id': item.get('ground_truth', {}).get('id')
    }
    papers_db[paper_id]['queries'].append(query_obj)
    
    # 2. Candidate Pool
    gt = item.get('ground_truth', {})
    cand_id = gt.get('id')
    
    if cand_id and cand_id not in papers_db[paper_id]['candidates']:
        cand_authors = gt.get('authors', [])
        
        # Lấy năm từ submission_date (Metadata)
        gt_date = gt.get('submission_date', '')
        gt_year = str(gt_date)[:4] if gt_date and len(str(gt_date)) >= 4 else ''
        
        papers_db[paper_id]['candidates'][cand_id] = {
            'cand_id': cand_id,
            'cand_title': normalize_text(gt.get('title', '')),
            'cand_authors': [normalize_text(a) for a in cand_authors],
            'cand_year': gt_year # <--- ĐÃ CÓ NĂM
        }

print(f"✅ Data transformed into {len(papers_db)} papers.")

Loaded 812 entries.
✅ Data transformed into 39 papers.


## 3. Load Trained Model
Load model SVM/RandomForest/XGBoost đã lưu từ bước Training.


In [71]:
try:
    model = joblib.load(MODEL_PATH)
    feature_names = joblib.load(FEATURE_NAME_PATH)
    print(f"✅ Model loaded: {type(model).__name__}")
    print(f"📝 Expected features: {len(feature_names)}")
except FileNotFoundError:
    print("⚠️ WARNING: Model file not found. Code will run with RANDOM SCORES for demonstration.")
    model = None
    feature_names = []


✅ Model loaded: XGBClassifier
📝 Expected features: 13


## 4. Ranking Pipeline & Prediction
Với mỗi bài báo:
1. Tạo cặp (Query, Candidate) cho tất cả ứng viên.
2. Tính feature.
3. Dự đoán xác suất match.
4. Lấy Top 5 ứng viên có điểm cao nhất.


In [72]:
global_mrr_sum = 0
global_query_count = 0

print("🚀 Starting Ranking Pipeline...")

for paper_id, data in papers_db.items():
    queries = data['queries']
    candidates_dict = data['candidates']
    candidates_list = list(candidates_dict.values())
    
    if not candidates_list:
        continue

    # Init Output Structure
    submission_data = {
        "partition": "test", 
        "groundtruth": {},
        "prediction": {}
    }
    
    # Loop qua từng query trong paper
    for query in tqdm(queries, desc=f"Paper {paper_id}", leave=False):
        bib_key = query['key']
        true_id = query['true_id']
        
        # Groundtruth
        submission_data['groundtruth'][bib_key] = true_id
        
        # Pairing & Feature Calc
        pairs = []
        for cand in candidates_list:
            row = {}
            row.update(query)
            row.update(cand)
            pairs.append(row)
            
        # Compute Features
        feats_list = [compute_pairwise_features(p) for p in pairs]
        df_feats = pd.DataFrame(feats_list)
        
        # Predict
        scores = []
        if model:
            # Ensure columns exist
            for col in feature_names:
                if col not in df_feats.columns: df_feats[col] = 0.0
            
            X_input = df_feats[feature_names]
            scores = model.predict_proba(X_input)[:, 1]
        else:
            scores = np.random.rand(len(pairs)) # Fallback
            
        # Ranking
        ranked_candidates = []
        for idx, score in enumerate(scores):
            ranked_candidates.append((candidates_list[idx]['cand_id'], score))
        
        ranked_candidates.sort(key=lambda x: x[1], reverse=True)
        top_5 = [x[0] for x in ranked_candidates[:5]]
        
        # Save Prediction
        submission_data['prediction'][bib_key] = top_5
        
        # Calc MRR
        if true_id in top_5:
            rank = top_5.index(true_id) + 1
            global_mrr_sum += 1.0 / rank
        else:
            global_mrr_sum += 0.0
            
        global_query_count += 1
        
    # Save JSON Output
    # Tên file pred.json nên đặt trong folder riêng của student/paper nếu nộp thật
    # Ở đây lưu dạng prefix_pred.json để dễ kiểm tra
    safe_pid = str(paper_id).replace('/', '_')
    save_path = os.path.join(OUTPUT_DIR, f"{safe_pid}_pred.json")
    with open(save_path, 'w', encoding='utf-8') as f:
        json.dump(submission_data, f, indent=4)

print("✅ Ranking completed for all papers.")


🚀 Starting Ranking Pipeline...


✅ Ranking completed for all papers.


## 5. Evaluation Report (Nhận xét)


In [73]:
final_mrr = global_mrr_sum / global_query_count if global_query_count > 0 else 0

print("="*40)
print("📊 FINAL EVALUATION REPORT")
print("="*40)
print(f"Papers Processed: {len(papers_db)}")
print(f"Total Queries:    {global_query_count}")
print(f"Metric MRR:       {final_mrr:.4f}")
print("="*40)

if final_mrr > 0.8:
    print("🌟 Nhận xét: Xuất sắc. Model xếp hạng reference rất chính xác.")
elif final_mrr > 0.5:
    print("👍 Nhận xét: Khá. Model đưa ra ứng viên đúng trong top 5 thường xuyên.")
else:
    print("⚠️ Nhận xét: Cần cải thiện. Kiểm tra lại Features hoặc Model.")

print(f"\n📁 File kết quả (pred.json) lưu tại: {os.path.abspath(OUTPUT_DIR)}")


📊 FINAL EVALUATION REPORT
Papers Processed: 39
Total Queries:    812
Metric MRR:       0.8349
🌟 Nhận xét: Xuất sắc. Model xếp hạng reference rất chính xác.

📁 File kết quả (pred.json) lưu tại: d:\Coding\School\Y3-K1\Intro2DS\DS - LAB 2\Milestone2_Project\23127011\notebooks\submission_output
